In [1]:
# Load data ke dalam data frame 
import pandas as pd

# Spesifikasi encoding diperlukan karena data tidak menggunakan UTF-8
df = pd.read_csv('scraping_avoskin-neg 340.csv', encoding='latin-1')

df.head()

,Review,Product,Rating,Username,Date
0,Kecewa banget. Saya pesan yang free pouch tapi...,Perfect Light Skin Protection (BLPXAVOSKIN MTS...,bintang 1,A***a,Lebih dari 1 tahun lalu
1,"Kali ini pengemasan tidak safe,, sepertinya tu...",Avoskin Hydrating Treatment Essence,bintang 1,Joshua,Lebih dari 1 tahun lalu
2,paket datangnya barang sudah pecah dan sy retu...,Avoskin Hydrating Treatment Essence,bintang 1,Fransisca,Lebih dari 1 tahun lalu
3,"ga ada sabun nya, ga sesuai deskripsi",Avoskin Perfect Hydrating Treatment Essence 100ml,bintang 1,A***y,Lebih dari 1 tahun lalu
4,ga sesuai harapan,Avoskin Miraculous Retinol Series,bintang 1,D***a,Lebih dari 1 tahun lalu


In [2]:
# Drop kolom yang tidak terpakai
df = df.drop(df.iloc[:,1:], axis=1)

# Cek data teratas
df.head()

,Review
0,Kecewa banget. Saya pesan yang free pouch tapi...
1,"Kali ini pengemasan tidak safe,, sepertinya tu..."
2,paket datangnya barang sudah pecah dan sy retu...
3,"ga ada sabun nya, ga sesuai deskripsi"
4,ga sesuai harapan


PREPROCESSING

In [3]:
# PROSES CASE FOLDING (menjadikan semua huruf di kolom Review menjadi huruf kecil)
df['Review'] = df['Review'].str.lower()

print('Case Folding Result: \n')
df.head()

Case Folding Result: 



,Review
0,kecewa banget. saya pesan yang free pouch tapi...
1,"kali ini pengemasan tidak safe,, sepertinya tu..."
2,paket datangnya barang sudah pecah dan sy retu...
3,"ga ada sabun nya, ga sesuai deskripsi"
4,ga sesuai harapan


In [4]:
# import hasil case folding menjadi csv
df.to_csv("casefolding_avoskin.csv")

In [5]:
# CLEANING
import re # regex library

def remove_tweet_special(text):
    # remove tab, new line, and back slice
    text = text.replace('\\t'," ").replace('\\n', " ").replace('\\u', " ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df['Review'] = df['Review'].apply(remove_tweet_special)

# remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['Review'] = df['Review'].apply(remove_number)

# remove punctuation and replace with space
def remove_punctuation(text):
    return re.sub(r'[.,]', ' ', text)

df['Review'] = df['Review'].apply(remove_punctuation)

# remove punctuation
def remove_symbol(text):
    # Menghapus simbol-simbol tidak standar dan menggantinya dengan spasi
    cleaned_text = re.sub(r'[^\w\s]', ' ', text)
    
    # Menghapus multiple whitespace
    cleaned_text = re.sub('\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

df['Review'] = df['Review'].apply(remove_symbol)

# Menentukan ambang batas panjang string acak
threshold_length = 20

# Fungsi untuk menghapus string acak berdasarkan panjangnya
def hapus_string_acak_dengan_panjang(text):
    # Memeriksa panjang string dan menghapus jika melebihi ambang batas
    return ' '.join(word for word in text.split() if len(word) <= threshold_length)

# Menggunakan fungsi untuk menghapus string acak pada kolom 'Text'
df['Review'] = df['Review'].apply(hapus_string_acak_dengan_panjang)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['Review'] = df['Review'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['Review'] = df['Review'].apply(remove_whitespace_multiple)

# remove single char
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['Review'] = df['Review'].apply(remove_single_char)

def remove_laughter(text):
    laughter_patterns = r'\b((ha)+h*|(he)+h*|(hi)+h*|(wk)+w*k*|(eh)+e*|(ah)+a*|(ih)+i*|(kw)+k*w*|(hem)+m*)\b'
    return re.sub(laughter_patterns, '', text, flags=re.IGNORECASE)

df['Review'] = df['Review'].apply(remove_laughter)

print('Cleaning Result : \n') 
print(df.head())

Cleaning Result : 

                                              Review
0  kecewa banget saya pesan yang free pouch tapi ...
1  kali ini pengemasan tidak safe sepertinya tutu...
2  paket datangnya barang sudah pecah dan sy retu...
3               ga ada sabun nya ga sesuai deskripsi
4                                  ga sesuai harapan


In [6]:
# import hasil cleaning menjadi csv
df.to_csv("cleaning_avoskin.csv")

In [7]:
# REMOVE DUPLICATE
df = df.drop_duplicates()
df = df.reset_index(drop=True)
# Menghapus baris yang kosong
df = df.dropna(subset=['Review'])
# Menghapus baris yang hanya berisi spasi atau whitespace
df = df[df['Review'].str.strip() != '']
df.head()

,Review
0,kecewa banget saya pesan yang free pouch tapi ...
1,kali ini pengemasan tidak safe sepertinya tutu...
2,paket datangnya barang sudah pecah dan sy retu...
3,ga ada sabun nya ga sesuai deskripsi
4,ga sesuai harapan


In [8]:
# import hasil remove duplicate menjadi csv
df.to_csv("removedup_avoskin.csv")

In [10]:
# NORMALIZATION
import pandas as pd
import re

slang_dictionary = pd.read_csv('../../colloquial-indonesian-lexicon2.csv')
slang_dict = pd.Series(slang_dictionary['formal'].values,index=slang_dictionary['slang']).to_dict()

slang_dictionary.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1.0,wow,elongasi,0,0
1,aminn,amin,1.0,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1.0,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1.0,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0.0,Birthday yg keberpa kak?,abreviasi,0,0


In [11]:
# Normalisasi kata menggunakan kamus colloquial-indonesian-lexicon2.csv
def Slangwords(text, slang_dict):
    for word in text.split():
        if word in slang_dict.keys():
            # menambahkan \b untuk menandakan batas kata di sekitar kata slang
            text = re.sub(r'\b{}\b'.format(re.escape(word)), slang_dict[word], text)
    
    text = re.sub('@[\w]+', '', text)
    return text

df['Review_norm'] = df['Review'].apply(lambda x: Slangwords(x, slang_dict))
print(df.head())

                                              Review  \
0  kecewa banget saya pesan yang free pouch tapi ...   
1  kali ini pengemasan tidak safe sepertinya tutu...   
2  paket datangnya barang sudah pecah dan sy retu...   
3               ga ada sabun nya ga sesuai deskripsi   
4                                  ga sesuai harapan   

                                         Review_norm  
0  kecewa banget saya pesan yang free pouch tapi ...  
1  kali ini pengemasan tidak safe sepertinya tutu...  
2  paket datangnya barang sudah pecah dan saya re...  
3       enggak ada sabun nya enggak sesuai deskripsi  
4                              enggak sesuai harapan  


In [12]:
# import hasil replace slang words menjadi csv
df.to_csv("normalization_avoskin.csv")

In [13]:
# PROSES TOKENIZING (word_tokenize() untuk memecah string kedalam tokens)

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Tokenizing
# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['review_tokens'] = df['Review_norm'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df.head())

Tokenizing Result : 

                                              Review  \
0  kecewa banget saya pesan yang free pouch tapi ...   
1  kali ini pengemasan tidak safe sepertinya tutu...   
2  paket datangnya barang sudah pecah dan sy retu...   
3               ga ada sabun nya ga sesuai deskripsi   
4                                  ga sesuai harapan   

                                         Review_norm  \
0  kecewa banget saya pesan yang free pouch tapi ...   
1  kali ini pengemasan tidak safe sepertinya tutu...   
2  paket datangnya barang sudah pecah dan saya re...   
3       enggak ada sabun nya enggak sesuai deskripsi   
4                              enggak sesuai harapan   

                                       review_tokens  
0  [kecewa, banget, saya, pesan, yang, free, pouc...  
1  [kali, ini, pengemasan, tidak, safe, sepertiny...  
2  [paket, datangnya, barang, sudah, pecah, dan, ...  
3  [enggak, ada, sabun, nya, enggak, sesuai, desk...  
4                          [e

In [14]:
# import hasil tokenizing menjadi csv
df.to_csv("tokenizing_avoskin.csv")

In [15]:
# PROSES FILTERING (Stopword Removal)
from nltk.corpus import stopwords

# get stopword indonesia from NLTK stopword
list_stopwords = stopwords.words('indonesian')

# convert list to dictionary
list_stopwords = set(list_stopwords)

# remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['review_tokens_SR'] = df['review_tokens'].apply(stopwords_removal) 

print(df.head())

                                              Review  \
0  kecewa banget saya pesan yang free pouch tapi ...   
1  kali ini pengemasan tidak safe sepertinya tutu...   
2  paket datangnya barang sudah pecah dan sy retu...   
3               ga ada sabun nya ga sesuai deskripsi   
4                                  ga sesuai harapan   

                                         Review_norm  \
0  kecewa banget saya pesan yang free pouch tapi ...   
1  kali ini pengemasan tidak safe sepertinya tutu...   
2  paket datangnya barang sudah pecah dan saya re...   
3       enggak ada sabun nya enggak sesuai deskripsi   
4                              enggak sesuai harapan   

                                       review_tokens  \
0  [kecewa, banget, saya, pesan, yang, free, pouc...   
1  [kali, ini, pengemasan, tidak, safe, sepertiny...   
2  [paket, datangnya, barang, sudah, pecah, dan, ...   
3  [enggak, ada, sabun, nya, enggak, sesuai, desk...   
4                          [enggak, sesuai, ha

In [16]:
# import hasil stopword removal menjadi csv
df.to_csv("stopword_avoskin.csv")

In [17]:
# PROSES LEMMATIZATION 
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['review_tokens_SR']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['review_tokens_stemmed'] = df['review_tokens_SR'].swifter.apply(get_stemmed_term)
print(df['review_tokens_stemmed'])
print(df.head())

1244
------------------------
kecewa : kecewa
banget : banget
pesan : pesan
free : free
pouch : pouch
dikirimnya : kirim
cotton : cotton
pad : pad
kaget : kaget
pas : pas
unboxing : unboxing
untung : untung
videokan : video
kabarin : kabarin
customer : customer
servicenya : servicenya
disuruh : suruh
isi : isi
googleform : googleform
dikirimkan : kirim
ulang : ulang
pouchnya : pouchnya
kabar : kabar
dikabari : kabar
stock : stock
habis : habis
sabar : sabar
mengikuti : ikut
prosedur : prosedur
service : service
berpikiran : pikir
positif : positif
ya : ya
kali : kali
pengemasan : emas
safe : safe
tutup : tutup
botol : botol
kencang : kencang
toner : toner
merembes : rembes
dus : dus
basah : basah
video : video
produk : produk
packaging : packaging
bagus : bagus
avoskin : avoskin
hujan : hujan
dibuka : buka
kosong : kosong
semoga : moga
tanggapan : tanggap
paket : paket
datangnya : datang
barang : barang
pecah : pecah
return : return
seller : seller
ikuti : ikut
komplain : komplain
pros

Pandas Apply: 100%|██████████| 278/278 [00:00<?, ?it/s]

0      [kecewa, banget, pesan, free, pouch, kirim, co...
1      [kali, emas, safe, tutup, botol, kencang, tone...
2      [paket, datang, barang, pecah, return, seller,...
3                        [sabun, nya, sesuai, deskripsi]
4                                        [sesuai, harap]
                             ...                        
274    [bara, selamat, customer, service, avoskin, to...
275                       [barang, bagus, sayang, layan]
276    [halo, kak, warna, produk, kuning, bening, kay...
277    [harga, kejar, diskon, doh, not, cheap, at, al...
278                    [ukur, produk, mahal, harga, gin]
Name: review_tokens_stemmed, Length: 278, dtype: object
                                              Review  \
0  kecewa banget saya pesan yang free pouch tapi ...   
1  kali ini pengemasan tidak safe sepertinya tutu...   
2  paket datangnya barang sudah pecah dan sy retu...   
3               ga ada sabun nya ga sesuai deskripsi   
4                                  ga

In [18]:
# import hasil preprocessing menjadi csv
df.to_csv("lemma_avoskin 340.csv")